In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
def fetch_pool_prices(pool_address, minutes = 400):
    end_timestamp = datetime.utcnow()
    start_timestamp = end_timestamp - timedelta(minutes = minutes)

    query = """{swaps(where:{pool: "%s", timestamp_gte: %d,
    timestamp_lte: %d},
    orderBy: timestamp, orderDirection: asc){
    timestamp
    amount0 
    amount1 
    }}""" % (pool_address, start_timestamp.timestamp(), end_timestamp.timestamp())

    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    response = requests.post(url, json={'query': query})
    data = response.json()

    swaps = data.get('data',()).get('swaps', [])

    df = pd.DataFrame(swaps)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s') #Change from UNIX to datetime
    df['price'] = abs(df['amount1'].astype(float)/df['amount0'].astype(float))

    return df

In [4]:
pool_address = '0x7baecE5d47f1BC5E1953FBE0E9931D54DAB6D810'.lower() #$TURBO/WETH pool address
fetch_pool_prices(pool_address)

AttributeError: 'tuple' object has no attribute 'get'

In [ ]:
def fetch_pool_prices_pagi(pool_address, minutes = 400):
    end_timestamp = datetime.utcnow()
    start_timestamp = end_timestamp - timedelta(minutes = minutes)

    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

    all_swaps = [] #Will store all iterations of queries
    last_id = "" 

    while True:
        query = """{swaps(first:100,where:{pool: "%s", timestamp_gte: %d,
        timestamp_lte: %d, id_gt: "%s"},
        orderBy: id, orderDirection: asc){
        id
        timestamp
        amount0 
        amount1 
        }}""" % (pool_address, start_timestamp.timestamp(), end_timestamp.timestamp(), last_id)

        response = requests.post(url, json={'query': query})
        data = response.json()
        swaps = data.get('data',()).get('swaps', [])
        if not swaps:
            break
        all_swaps.extend(swaps)
        last_id = swaps[-1]['id']

        if len(swaps) < 100:
            break
       
    df = pd.DataFrame(all_swaps)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s') #Change from UNIX to datetime
    df['price'] = abs(df['amount1'].astype(float)/df['amount0'].astype(float))

    return df

In [ ]:
df_x = fetch_pool_prices_pagi(pool_address,minutes=5000)

df_x.set_index('timestamp').price.plot()